In [ ]:
import os
import pandas as pd
import numpy as np

from utils.hp_functions import get_dset_hp # Helper functions to load default hyper-parameters

In [ ]:
tasks = {'office-home': ['AC', 'AP', 'AR', 'CA', 'CP', 'CR', 'PA', 'PC', 'PR', 'RA', 'RC', 'RP'],
         'visda': ['SR', 'RS']
        }
methods = ['s. only', 'pada', 'safn', 'ba3us', 'ar', 'jumbot', 'mpot']

In [ ]:
def get_tables(logs_folder, dataset, methods, hp):
    table = {}
    _, domains = get_dset_hp(dataset, '')
    seeds = [2020,2021,2022]
    net = 'ResNet50'
    count = 0
    for hp_metric in ['oracle', '1shot_acc', '50random_acc', '100random_acc', 's_acc', 'ent', 'dev_svm', 'snd']:
        table[hp_metric] = {}
        for sel_metric in ['final', 'oracle', '1shot_acc', '50random_acc', '100random_acc', 's_acc', 'ent', 'dev_svm', 'snd']:
            table[hp_metric][sel_metric] = np.zeros((len(methods),len(domains)*(len(domains)-1), len(seeds)))
        for ix, method in enumerate(methods):
            jx = 0
            for source_domain in domains:
                for target_domain in domains:
                    if source_domain != target_domain:
                        task = source_domain[0].upper() + target_domain[0].upper()
                        for kx, seed in enumerate(seeds):
                            output_dir = os.path.join(logs_folder, method, net, dataset, task)
                            for key in hp[method][hp_metric]:
                                output_dir = os.path.join(output_dir, f'{key}_{hp[method][hp_metric][key]}')
                            path_results = os.path.join(output_dir, f"seed_{seed}", 'run_0', 'results.npy')
                            if os.path.exists(path_results):
                                results = np.load(path_results, allow_pickle=True).item()
                                t_acc = np.array(results['t_acc'])
                                table[hp_metric]['final'][ix,jx,kx] = t_acc[-1]
                                table[hp_metric]['oracle'][ix,jx,kx] = t_acc.max()
                                for temp_metric in ['ent', 'dev_svm']:
                                    table[hp_metric][temp_metric][ix,jx,kx] = t_acc[np.array(results[temp_metric]).argmin()]
                                for temp_metric in ['snd', '1shot_acc', '100random_acc', 's_acc']:
                                    table[hp_metric][temp_metric][ix,jx,kx] = t_acc[np.array(results[temp_metric]).argmax()]
                                if hp_metric in ['50random_acc']:
                                    for temp_metric in ['50random_acc']:
                                        if temp_metric in results.keys():
                                            table[hp_metric][temp_metric][ix,jx,kx] = t_acc[np.array(results[temp_metric]).argmax()]
#                                         else:
#                                             count += 1
#                                             print(f'Missing {hp_metric} on {task} for {method} with seed {seed}')                                                
                        jx += 1
    return table

## Gather tables for the main results

In [ ]:
tables = {}
logs_folder = 'logs_hp_chosen'
methods = ['source_only_plus', 'pada', 'safn', 'ba3us', 'ar', 'jumbot', 'mpot']
for dataset in ['office-home', 'visda']:
    hp = np.load('results/hp_chosen.npy', allow_pickle=True).item()
    hp = hp[dataset]
    tables[dataset] = get_tables(logs_folder, dataset, methods, hp)

## Gather tables with feature normalization in bottleneck layer

In [ ]:
tables_radius = {}
logs_folder = 'logs_hp_chosen_radius'
methods = ['ar']
for dataset in ['office-home']:
    hp = np.load('results/hp_chosen_radius.npy', allow_pickle=True).item()
    hp = hp[dataset]
    tables_radius[dataset] = get_tables(logs_folder, dataset, methods, hp)

## Gather tables with nonlinear bottleneck layer

In [ ]:
tables_nonlinear = {}
logs_folder = 'logs_hp_chosen_nonlinear'
methods = ['safn']
for dataset in ['office-home']:
    hp = np.load('results/hp_chosen_nonlinear.npy', allow_pickle=True).item()
    hp = hp[dataset]
    tables_nonlinear[dataset] = get_tables(logs_folder, dataset, methods, hp)

Counting number of models trained for main results.

In [ ]:
hp = np.load('results/hp_chosen.npy', allow_pickle=True).item()
hp_metrics = ['oracle', '1shot_acc', '100random_acc', 's_acc', 'ent', 'dev_svm', 'snd']
for dataset in ['office-home', 'visda']:
    total = 0
    for method in ['pada', 'safn', 'ar', 'ba3us', 'jumbot', 'mpot']:
        hp_values = hp[dataset][method]['oracle'].keys()
        table = np.zeros((len(hp_metrics), len(hp_values)))
        for ix, metric in enumerate(hp_metrics):
            for jx, param in enumerate(hp_values):
                table[ix,jx] = hp[dataset][method][metric][param]
        count = np.unique(table,axis=0).shape[0]*3*len(tasks[dataset])
        print(f'For {method} we trained {count} models.')
        total += count
    print(f'For {dataset} we trained a total {total} models.')

## Reported results

We gather from the original papers the reported results for each method.

In [ ]:
reported = {'office-home': {}, 'visda': {}}

reported['office-home']['s. only'] = [46.33, 67.51, 75.87, 59.14, 59.94, 62.73, 58.22, 41.79, 74.88, 67.40, 48.18, 74.17]
reported['office-home']['ar_linear'] = [62.13, 79.22, 89.12, 73.92, 75.57, 84.37, 78.42, 61.91, 87.85, 82.19, 65.37, 85.27] #Avg  77.11
reported['office-home']['ar'] = [67.40, 85.32, 90.00, 77.32, 70.59, 85.15, 78.97, 64.78, 89.51, 80.44, 66.21, 86.44] #Avg  78.29
reported['office-home']['ba3us'] = [60.62, 83.16, 88.39, 71.75, 72.79, 83.40, 75.45, 61.59, 86.53, 79.25, 62.80, 86.05] #Avg 75.98
#BA3US stds [0.45, 0.12, 0.19, 0.19, 1.10, 0.59, 0.19, 0.37, 0.22, 0.65, 0.51, 0.26]
reported['office-home']['safn'] = [58.93, 76.25, 81.42, 70.43, 72.97, 77.78, 72.36, 55.34, 80.4, 75.81, 60.42, 79.92] #Avg 71.83
#SAFN stds [0.50, 0.33, 0.27, 0.46, 1.39, 0.52, 0.31, 0.46, 0.78, 0.37, 0.83, 0.20]
reported['office-home']['pada'] = [51.95, 67, 78.74, 52.16, 53.78, 59.03, 52.61, 43.22, 78.79, 73.73, 56.6, 77.09] #Avg  62.06
reported['office-home']['jumbot'] = [62.7, 77.5, 84.4, 76.0, 73.3, 80.5, 74.7, 60.8, 85.1, 80.2, 66.5, 83.9] #Avg  75.5
reported['office-home']['mpot'] = [64.60, 80.62, 87.17, 76.43, 77.61, 83.58, 77.07, 63.74, 87.63, 81.42, 68.50, 87.38] #Avg  77.98

reported['visda']['s. only'] = [45.26, 64.28] # Avg 54.77
reported['visda']['ba3us'] = [69.86, 67.56] # Avg 68.71
reported['visda']['pada'] = [53.53, 76.50] # Avg 65.01
reported['visda']['safn'] = [67.65, '-']
reported['visda']['ar_linear'] = [85.30, 74.82] # Avg  80.09
reported['visda']['ar'] = [88.75, 78.52] # Avg  83.62

## Code to generate Table 5

In [ ]:
methods = ['s. only', 'pada', 'safn', 'ba3us', 'ar', 'jumbot', 'mpot']

In [ ]:
dataset = 'office-home'
table = ['\\begin{table}[t!]\\centering']
table.append('\\resizebox{\\textwidth}{!}{')
temp = '\\begin{tabular}{c'
for i in range(len(tasks)+1):
    temp += '@{\hskip 3pt}|c'
temp += '}'
table.append(temp)
temp = '\\textsc{Method}'
for task in tasks[dataset]:
    temp += f' & {task}'
temp += ' & Avg\\\\'
table.append(temp)
table.append('\\midrule')
for ix, method in enumerate(methods):
    if ix == 0:
        pass
    else:
        table.append('\\midrule\\midrule')
    if method in ['safn', 'ar']:
        temp = '\\textsc{'+method+'}$^{\\dagger *}$'
    else:
        temp = '\\textsc{'+method+'}$^\\dagger$'
    if method == 'ar':
        mean = reported[dataset]['ar_linear']
    else:
        if method in reported[dataset].keys():
            mean = reported[dataset][method]
        else:
            mean = np.zeros(len(reported[dataset]['s. only']))
    for m in mean:
        temp += f' & {m:.2f}'
    mean = np.mean(mean)
    temp += f' & {np.mean(mean):.2f}'
    temp += '\\\\'
    table.append(temp)
    if method == 'safn':
        temp = '\\textsc{'+method+'}* (Ours)'
        mean = tables_nonlinear[dataset]['oracle']['oracle'].mean(axis=2)[0]*100
        stds = tables_nonlinear[dataset]['oracle']['oracle'].std(axis=2)[0]*100
        for m,std in zip(mean,stds):
            temp += f' & {m:.2f}'
        mean = tables_nonlinear[dataset]['oracle']['oracle'].mean(axis=1)[0]*100
        temp += f' & {np.mean(mean):.2f}'
        temp += '\\\\'
        table.append(temp)
    elif method == 'ar':
        temp = '\\textsc{'+method+'}* (Ours)'
        mean = tables_radius[dataset]['oracle']['oracle'].mean(axis=2)[0]*100
        stds = tables_radius[dataset]['oracle']['oracle'].std(axis=2)[0]*100
        for m,std in zip(mean,stds):
            temp += f' & {m:.2f}'
        mean = tables_radius[dataset]['oracle']['oracle'].mean(axis=1)[0]*100
        temp += f' & {np.mean(mean):.2f}'
        temp += '\\\\'
        table.append(temp)


    temp = '\\textsc{'+method+'} (Ours)'
    mean = tables[dataset]['oracle']['oracle'].mean(axis=2)[ix]*100
    stds = tables[dataset]['oracle']['oracle'].std(axis=2)[ix]*100
    for m,std in zip(mean,stds):
        temp += f' & {m:.2f}'
    mean = tables[dataset]['oracle']['oracle'].mean(axis=1)[ix]*100
    temp += f' & {np.mean(mean):.2f}'
    temp += '\\\\'
    table.append(temp)

table.append('\\end{tabular}}')
table.append('\\caption{Comparison between reported ($\\dagger$) accuracies on partial \\textsc{office-home} from published methods with our implementation using the \\textsc{oracle} model selection strategy. * denotes different bottleneck architectures.}')
table.append('\\label{table:office_home_comparison_reported}')
table.append('\\end{table}')
print('\n'.join(table))

## Code to generate Table 12

In [ ]:
dataset = 'visda'
table = ['\\begin{table}[t!]\\centering']
temp = '\\begin{tabular}{c'
for i in range(len(tasks)+1):
    temp += '@{\hskip 3pt}|c'
temp += '}'
table.append(temp)
temp = '\\textsc{Algorithm}'
for task in tasks[dataset]:
    temp += f' & {task}'
temp += ' & Avg\\\\'
table.append(temp)
table.append('\\midrule')
for ix, method in enumerate(methods):
    if method == 'gamma':
        pass
    else:
        if ix == 0:
            pass
        else:
            table.append('\\midrule\\midrule')
        temp = '\\textsc{'+method+'}$^\\dagger$'
        if method == 'ar':
            temp = '\\textsc{'+method+'}$^{\\dagger *}$'
            mean = reported[dataset]['ar_linear']
        else:
            if method in reported[dataset].keys():
                mean = reported[dataset][method]
            else:
                mean = np.zeros(len(reported[dataset]['s. only']))
        mean_task = None
        for m in mean:
            if isinstance(m,str) or m == 0.0:
                temp += ' & -'
                mean_task = 'na'
            else:
                temp += f' & {m:.2f}'
        if mean_task == 'na':
            temp += ' & -'
        else:
            temp += f' & {np.mean(mean):.2f}'
        temp += '\\\\'
        table.append(temp)
        temp = '\\textsc{'+method+'} (Ours)'
        mean = tables[dataset]['oracle']['oracle'].mean(axis=2)[ix]*100
        stds = tables[dataset]['oracle']['oracle'].std(axis=2)[ix]*100
        for m,std in zip(mean,stds):
            temp += f' & {m:.2f}'
        mean = tables[dataset]['oracle']['oracle'].mean(axis=1)[ix]*100
        temp += f' & {np.mean(mean):.2f}'
        temp += '\\\\'
        table.append(temp)


table.append('\\end{tabular}')
table.append('\\caption{Comparison between reported ($\\dagger$) accuracies on partial \\textsc{visda} from published methods with our implementation using the \\textsc{oracle} model selection strategy. * denotes different bottleneck architectures.}')
table.append('\\label{table:visda_comparison_reported}')
table.append('\\end{table}')
print('\n'.join(table))

## Code to generate Table 1

In [ ]:
hp_metrics = {'s_acc': 's-acc', 'ent': 'ent', 'dev_svm': 'dev', 'snd': 'snd', '1shot_acc': '1-shot',
              '100random_acc': '100-rnd', 'oracle': 'oracle'}
table = ['\\begin{table}[t!]\\centering']
table.append('\\resizebox{\\textwidth}{!}{')
temp = '\\begin{tabular}{c|@{\hskip 3pt}c|'
for method in methods:
    temp += '@{\hskip 3pt}|@{\hskip 3pt}c'
temp += '}'
table.append(temp)

temp = '\\textsc{Dataset} & Model Selection'
for method in methods:
    temp += ' & \\textsc{'+method+'}'
temp += '\\\\'
table.append(temp)

for dataset in ['office-home', 'visda']:
    table.append('\\midrule\\midrule')
    aux_table = np.zeros((len(methods), len(hp_metrics)))
    aux_table_stds = np.zeros((len(methods), len(hp_metrics)))
    for ix, method in enumerate(methods):
        for jx, metric in enumerate(hp_metrics):
            aux_table[ix,jx] = tables[dataset][metric][metric].mean(axis=1).mean(axis=1)[ix]*100
            aux_table_stds[ix,jx] = tables[dataset][metric][metric].mean(axis=1).std(axis=1)[ix]*100
    new_table = np.zeros((len(methods),3))
    new_table_stds = np.zeros((len(methods),3))
    new_table[:,0] = aux_table[:,:4].min(axis=1)
    new_table_stds[:,0] = aux_table_stds[np.arange(len(methods)),aux_table[:,:4].argmin(axis=1)]
    new_table[:,1] = aux_table[:,:4].max(axis=1)
    new_table_stds[:,1] = aux_table_stds[np.arange(len(methods)),aux_table[:,:4].argmax(axis=1)]
    new_table[:,2] = aux_table[:,-1]
    new_table_stds[:,2] = aux_table_stds[:,-1]
    new_table = new_table.transpose()
    new_table_stds = new_table_stds.transpose()
    temp = '\\multirow{2}{*}{\\textsc{'+dataset+'}} & Worst (w/o target labels)'
    worse_selection = ['(\\textsc{'+list(hp_metrics.values())[i]+'})' for i in aux_table[:,:4].argmin(axis=1)]
    for i in range(len(methods)):
        temp += f' & {new_table[0,i]:.2f} ({new_table[0,i]-new_table[2,i]:.2f})'
    temp += '\\\\'
    table.append(temp)
    temp = ' & Best (w/o target labels)'
    best_selection = ['(\\textsc{'+list(hp_metrics.values())[i]+'})' for i in aux_table[:,:4].argmax(axis=1)]
    for i in range(len(methods)):
        temp += f' & {new_table[1,i]:.2f} ({new_table[1,i]-new_table[2,i]:.2f})'
    temp += '\\\\'
    table.append(temp)
    temp = ' & \\textsc{{oracle}}'
    for i in range(len(methods)):
        temp += f' & {new_table[2,i]:.2f}'
    temp += '\\\\'
    table.append(temp)
table.append('\\end{tabular}}')
table.append('\\caption{Task accuracy average computed over three different seeds (2020, 2021, 2022) on Partial \\textsc{office-home} and Partial-\\textsc{visda}. For each dataset and PDA method, we display the results of the worst and best performing model selection that do not use target labels as well as the \\textsc{oracle} model selection strategy. All results can be found in Table \\ref{table:overall_results}.}')
table.append('\\label{table:small_overall_results}')
table.append('\\end{table}')
print('\n'.join(table))

## Code to generate Table 6

In [ ]:
table = ['\\begin{table}[t!]\\centering']
table.append('\\resizebox{\\textwidth}{!}{')
temp = '\\begin{tabular}{c|@{\hskip 3pt}c|'
for metric in hp_metrics:
    temp += '@{\hskip 3pt}|c'
temp += '}'
table.append(temp)
temp = '\\textsc{Dataset} & \\textsc{Method}'
for metric in hp_metrics:
    temp += ' & \\textsc{'+hp_metrics[metric]+'}'
temp += '\\\\'
table.append(temp)
for kx, dataset in enumerate(['office-home', 'visda']):
    table.append('\\midrule\\midrule')
    aux_table = np.zeros((len(methods), len(hp_metrics)))
    for ix, method in enumerate(methods):
        for jx, metric in enumerate(hp_metrics):
            aux_table[ix,jx] = tables[dataset][metric][metric].mean(axis=1).mean(axis=1)[ix]*100
    max_ix = aux_table.argmax(axis=0)
    for ix, method in enumerate(methods):
        if ix == 0:
            temp = '\\multirow{7}{*}{\\textsc{'+dataset+'}} & \\textsc{'+method+'} '
        else:
            temp = ' & \\textsc{'+method+'} '
        for jx, metric in enumerate(hp_metrics):
            mean = tables[dataset][metric][metric].mean(axis=1).mean(axis=1)[ix]*100
            std = tables[dataset][metric][metric].mean(axis=1).std(axis=1)[ix]*100
#             temp += f'& {mean:.2f}$\\:\\pm\\:${std:.1f}'
            if ix == max_ix[jx]:
                temp += '& \\textbf{'+f'{mean:.2f}$\\pm${std:.1f}'+'}'
            else:
                temp += f'& {mean:.2f}$\\pm${std:.1f}'
        temp += '\\\\'
        table.append(temp)    
table.append('\\end{tabular}}')
table.append('\\caption{Task accuracy average for the different PDA methods and model selection strategy pairs on Partial Office-Home and Partial VisDA. The average is computed over three difference seeds (2020, 2021, 2022).}')
table.append('\\label{table:overall_results}')
table.append('\\end{table}')
print('\n'.join(table))
print('\n')

## Code to generate Table 11

In [ ]:
dataset = 'office-home'
table = ['\\begin{table}[t!]\\centering']
if dataset == 'office-home':
    table.append('\\resizebox{\\textwidth}{!}{')
temp = '\\begin{tabular}{c'
for i in range(len(tasks[dataset])+2):
    temp += '@{\hskip 3pt}|{\hskip 3pt}c'
temp += '}'
table.append(temp)
temp = '\\textsc{Metric} & \\textsc{Method}'
for task in tasks[dataset]:
    temp += f' & {task}'
temp += ' & Avg\\\\'
table.append(temp)

for metric in hp_metrics:
    table.append('\\midrule\\midrule')

    aux_table = np.zeros((len(methods), len(tasks[dataset])+1))
    for ix, method in enumerate(methods):
        aux_table[ix,:-1] = tables[dataset][metric][metric].mean(axis=2)[ix]*100
    aux_table[:,-1] = aux_table[:,:-1].mean(axis=1)
    max_ix = aux_table.argmax(axis=0)
    for ix, method in enumerate(methods):
        if ix == 0:
            temp = '\\multirow{'+str(len(methods))+'}{*}{\\textsc{'+hp_metrics[metric]+'}}'
        else:
            temp = ''
        temp += ' & \\textsc{'+method+'}'
        mean = tables[dataset][metric][metric].mean(axis=2)[ix]*100
        stds = tables[dataset][metric][metric].std(axis=2)[ix]*100
        jx = 0
        for m,std in zip(mean,stds):
            if ix == max_ix[jx]:
                temp += ' & \\textbf{'+f'{m:.2f}$\\:\\pm\\:${std:.1f}'+'}'
            else:
                temp += f' & {m:.2f}$\\:\\pm\\:${std:.1f}'
            jx += 1

        mean = tables[dataset][metric][metric].mean(axis=1)[ix]*100
        if ix == max_ix[jx]:
            temp += ' & \\textbf{'+f'{np.mean(mean):.2f}$\\:\\pm\\:${np.std(mean):.1f}'+'}'
        else:
            temp += f' & {np.mean(mean):.2f}$\\:\\pm\\:${np.std(mean):.1f}'
        temp += '\\\\'
        table.append(temp)
table.append('\\end{tabular}}')
table.append('\\caption{Average accuracy of different PDA methods based on different model selection strategies on the 12 tasks of Partial \\textsc{office-home}. Average is done over three seeds (2020, 2021, 2022). Best results in \\textbf{bold}.}')
table.append('\\label{tab:office_home_results_tasks_with_stds}')
table.append('\\end{table}')
print('\n'.join(table))
print('\n')


## Code to generate Table 7

In [ ]:
dataset = 'visda'
table = ['\\begin{table}[t!]\\centering']
table.append('\\resizebox{\\textwidth}{!}{')
if dataset == 'office-home':
    table.append('\\resizebox{\\textwidth}{!}{')
temp = '\\begin{tabular}{c'
for i in range(len(hp_metrics)+1):
    temp += '@{\hskip 3pt}|c'
temp += '}'
table.append(temp)
temp = '\\textsc{Task} & \\textsc{Method}'
for metric in hp_metrics:
    temp += f' & '+'\\textsc{'+hp_metrics[metric]+'}'
temp += '\\\\'
table.append(temp)

for jx, task in enumerate(tasks[dataset]):
    table.append('\\midrule\\midrule')

    aux_table = np.zeros((len(methods),len(hp_metrics)))
    for i, method in enumerate(methods):
        for j, metric in enumerate(hp_metrics):
            aux_table[i,j] = tables[dataset][metric][metric][:,jx,:].mean(axis=1)[i]*100
    max_ix = aux_table.argmax(axis=0)

    for ix, method in enumerate(methods):
        if ix == 0:
            temp = '\\multirow{'+str(len(methods))+'}{*}{'+task+'}'
        else:
            temp = ''
        temp += ' & \\textsc{'+method+'}'
        for j, metric in enumerate(hp_metrics):
            mean = tables[dataset][metric][metric][:,jx,:].mean(axis=1)[ix]*100
            std = tables[dataset][metric][metric][:,jx,:].std(axis=1)[ix]*100
            if ix == max_ix[j]:
                temp += ' & \\textbf{'+f'{mean:.2f}$\\:\\pm\\:${std:.1f}'+'}'
            else:
                temp += f' & {mean:.2f}$\\:\\pm\\:${std:.1f}'
        temp += '\\\\'
        table.append(temp)
table.append('\\midrule\\midrule')

aux_table = np.zeros((len(methods),len(hp_metrics)))
for i, method in enumerate(methods):
    for j, metric in enumerate(hp_metrics):
        aux_table[i,j] = tables[dataset][metric][metric].mean(axis=1).mean(axis=1)[i]*100
max_ix = aux_table.argmax(axis=0)
for ix, method in enumerate(methods):
    if ix == 0:
        temp = '\\multirow{'+str(len(methods))+'}{*}{Avg}'
    else:
        temp = ''
    temp += ' & \\textsc{'+method+'}'
    for j, metric in enumerate(hp_metrics):
        mean = tables[dataset][metric][metric].mean(axis=1).mean(axis=1)[ix]*100
        std = tables[dataset][metric][metric].mean(axis=1).std(axis=1)[ix]*100
        if ix == max_ix[j]:
            temp += ' & \\textbf{'+f'{mean:.2f}$\\:\\pm\\:${std:.1f}'+'}'
        else:
            temp += f' & {mean:.2f}$\\:\\pm\\:${std:.1f}'
    temp += '\\\\'
    table.append(temp)
table.append('\\end{tabular}}')
table.append('\\caption{Accuracy of different PDA methods based on different model selection strategies on the 2 Partial \\textsc{visda} tasks. Average is done over three seeds (2020, 2021, 2022). Best results in \\textbf{bold}.}')
table.append('\\label{table:visda_results_tasks}')
table.append('\\end{table}')
print('\n'.join(table))

## Code to generate Table 13

In [ ]:
hp_metrics = {'s_acc': 's-acc', 'ent': 'ent', 'dev_svm': 'dev', 'snd': 'snd', '1shot_acc': '1-shot',
              '50random_acc': '50-rnd', '100random_acc': '100-rnd', 'oracle': 'oracle'}
table = ['\\begin{table}[t!]\\centering']
table.append('\\resizebox{\\textwidth}{!}{')
temp = '\\begin{tabular}{c|@{\hskip 3pt}c|'
for metric in hp_metrics:
    temp += '@{\hskip 3pt}|c'
temp += '}'
table.append(temp)
temp = '\\textsc{Dataset} & \\textsc{Method}'
for metric in hp_metrics:
    temp += ' & \\textsc{'+hp_metrics[metric]+'}'
temp += '\\\\'
table.append(temp)
for kx, dataset in enumerate(['office-home', 'visda']):
    table.append('\\midrule\\midrule')
    aux_table = np.zeros((len(methods), len(hp_metrics)))
    for ix, method in enumerate(methods):
        for jx, metric in enumerate(hp_metrics):
            aux_table[ix,jx] = tables[dataset][metric][metric].mean(axis=1).mean(axis=1)[ix]*100
    max_ix = aux_table.argmax(axis=0)
    for ix, method in enumerate(methods):
        if ix == 0:
            temp = '\\multirow{7}{*}{\\textsc{'+dataset+'}} & \\textsc{'+method+'} '
        else:
            temp = ' & \\textsc{'+method+'} '
        for jx, metric in enumerate(hp_metrics):
            mean = tables[dataset][metric][metric].mean(axis=1).mean(axis=1)[ix]*100
            std = tables[dataset][metric][metric].mean(axis=1).std(axis=1)[ix]*100
#             temp += f'& {mean:.2f}$\\:\\pm\\:${std:.1f}'
            if ix == max_ix[jx]:
                temp += '& \\textbf{'+f'{mean:.2f}$\\pm${std:.1f}'+'}'
            else:
                temp += f'& {mean:.2f}$\\pm${std:.1f}'
        temp += '\\\\'
        table.append(temp)    
table.append('\\end{tabular}}')
table.append('\\caption{Task accuracy average for the different PDA methods and model selection strategy pairs on Partial Office-Home and Partial VisDA. The average is computed over three difference seeds (2020, 2021, 2022).}')
table.append('\\label{table:overall_results}')
table.append('\\end{table}')
print('\n'.join(table))
print('\n')